<a href="https://colab.research.google.com/github/a0972340800/100-4/blob/master/Pytorch/Pytorch_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Pytorch Tutorial**


In [ ]:
import torch

**1. Pytorch Documentation Explanation with torch.max**



In [ ]:
x = torch.randn(4,5)
y = torch.randn(4,5)
z = torch.randn(4,5)
print(x)
print(y)
print(z)

In [ ]:
# 1. max of entire tensor (torch.max(input) → Tensor)
m = torch.max(x)
print(m)

In [ ]:
# 2. max along a dimension (torch.max(input, dim, keepdim=False, *, out=None) → (Tensor, LongTensor))
m, idx = torch.max(x,0)
print(m)
print(idx)

In [ ]:
# 2-2
m, idx = torch.max(input=x,dim=0)
print(m)
print(idx)

In [ ]:
# 2-3
m, idx = torch.max(x,0,False)
print(m)
print(idx)

In [ ]:
# 2-4
m, idx = torch.max(x,dim=0,keepdim=True)
print(m)
print(idx)

In [ ]:
# 2-5
p = (m,idx)
torch.max(x,0,False,out=p)
print(p[0])
print(p[1])


In [ ]:
# 2-6
p = (m,idx)
torch.max(x,0,False,p)
print(p[0])
print(p[1])

In [ ]:
# 2-7
m, idx = torch.max(x,True)

In [ ]:
# 3. max(choose max) operators on two tensors (torch.max(input, other, *, out=None) → Tensor)
t = torch.max(x,y)
print(t)

**2. Common errors**



The following code blocks show some common errors while using the torch library. First, execute the code with error, and then execute the next code block to fix the error. You need to change the runtime to GPU.


In [ ]:
import torch

In [ ]:
# 1. different device error
model = torch.nn.Linear(5,1).to("cuda:0")
x = torch.Tensor([1,2,3,4,5]).to("cpu")
y = model(x)

RuntimeError: ignored

In [ ]:
# 1. different device error (fixed)
x = torch.Tensor([1,2,3,4,5]).to("cuda:0")
y = model(x)
print(y.shape)

torch.Size([1])


In [ ]:
# 2. mismatched dimensions error
x = torch.randn(4,5)
y= torch.randn(5,4)
z = x + y

RuntimeError: ignored

In [ ]:
# 2. mismatched dimensions error (fixed)
y= y.transpose(0,1)
z = x + y
print(z.shape)

torch.Size([4, 5])


In [ ]:
# 3. cuda out of memory error
import torch
import torchvision.models as models
resnet18 = models.resnet18().to("cuda:0") # Neural Networks for Image Recognition
data = torch.randn(2048,3,244,244) # Create fake data (512 images)
out = resnet18(data.to("cuda:0")) # Use Data as Input and Feed to Model
print(out.shape)


RuntimeError: ignored

In [ ]:
# 3. cuda out of memory error (fixed)
for d in data:
  out = resnet18(d.to("cuda:0").unsqueeze(0))
print(out.shape)

torch.Size([1, 1000])


In [ ]:
# 4. mismatched tensor type
import torch.nn as nn
L = nn.CrossEntropyLoss()
outs = torch.randn(5,5)
labels = torch.Tensor([1,2,3,4,0])
lossval = L(outs,labels) # Calculate CrossEntropyLoss between outs and labels

RuntimeError: ignored

In [ ]:
# 4. mismatched tensor type (fixed)
labels = labels.long()
lossval = L(outs,labels)
print(lossval)

tensor(2.6215)


**3. More on dataset and dataloader**


A dataset is a cluster of data in a organized way. A dataloader is a loader which can iterate through the data set.

Let a dataset be the English alphabets "abcdefghijklmnopqrstuvwxyz"

In [ ]:
dataset = "abcdefghijklmnopqrstuvwxyz"

A simple dataloader could be implemented with the python code "for"

In [ ]:
for datapoint in dataset:
  print(datapoint)

When using the dataloader, we often like to shuffle the data. This is where torch.utils.data.DataLoader comes in handy. If each data is an index (0,1,2...) from the view of torch.utils.data.DataLoader, shuffling can simply be done by shuffling an index array.



torch.utils.data.DataLoader will need two imformation to fulfill its role. First, it needs to know the length of the data. Second, once torch.utils.data.DataLoader outputs the index of the shuffling results, the dataset needs to return the corresponding data.

Therefore, torch.utils.data.Dataset provides the imformation by two functions, `__len__()` and `__getitem__()` to support torch.utils.data.Dataloader

In [ ]:
import torch
import torch.utils.data
class ExampleDataset(torch.utils.data.Dataset):
  def __init__(self):
    self.data = "abcdefghijklmnopqrstuvwxyz"

  def __getitem__(self,idx): # if the index is idx, what will be the data?
    return self.data[idx]

  def __len__(self): # What is the length of the dataset
    return len(self.data)

dataset1 = ExampleDataset() # create the dataset
dataloader = torch.utils.data.DataLoader(dataset = dataset1,shuffle = True,batch_size = 1)
for datapoint in dataloader:
  print(datapoint)

A simple data augmentation technique can be done by changing the code in `__len__()` and `__getitem__()`. Suppose we want to double the length of the dataset by adding in the uppercase letters, using only the lowercase dataset, you can change the dataset to the following.

In [ ]:
import torch.utils.data
class ExampleDataset(torch.utils.data.Dataset):
  def __init__(self):
    self.data = "abcdefghijklmnopqrstuvwxyz"

  def __getitem__(self,idx): # if the index is idx, what will be the data?
    if idx >= len(self.data): # if the index >= 26, return upper case letter
      return self.data[idx%26].upper()
    else: # if the index < 26, return lower case, return lower case letter
      return self.data[idx]

  def __len__(self): # What is the length of the dataset
    return 2 * len(self.data) # The length is now twice as large

dataset1 = ExampleDataset() # create the dataset
dataloader = torch.utils.data.DataLoader(dataset = dataset1,shuffle = True,batch_size = 1)
for datapoint in dataloader:
  print(datapoint)

# Homework 1: COVID-19 Cases Prediction (Regression)

Download Data

In [4]:
tr_path = 'covid.train.csv'  # path to training data
# 這行程式碼將字串 'covid.train.csv' 賦值給變數 tr_path，表示訓練數據檔案的路徑或名稱。功能：用於記錄訓練檔案的檔案名稱，稍後可以使用這個變數來讀取或操作該檔案。
tt_path = 'covid.test.csv'   # path to testing data
# 這行程式碼將字串 'covid.test.csv' 賦值給變數 tt_path，表示測試數據檔案的路徑或名稱。功能：用於記錄測試檔案的檔案名稱，稍後可以使用這個變數來讀取或操作該檔案。
!gdown --id '19CCyCgJrUxtvgZF53vnctJiOJ23T5mqF' --output covid.train.csv # 這行程式碼使用了 gdown 工具從 Google Drive 上下載檔案，並將其存儲為名為 covid.train.csv 的檔案。
# !：在 Jupyter Notebook 或類似的互動式 Python 環境中，! 表示執行系統指令。這裡執行的是 gdown 命令。
# gdown：這是一個 Python 外部的工具，用於通過 Google Drive 檔案 ID 下載檔案。
# --id '19CCyCgJrUxtvgZF53vnctJiOJ23T5mqF'：指定要下載的檔案的 Google Drive ID。這個 ID 是唯一的，用於識別 Google Drive 中的特定檔案。
# --output covid.train.csv：指定下載後的檔案名稱為 covid.train.csv，並將其存儲到當前的工作目錄中。
!gdown --id '1CE240jLm2npU-tdz81-oVKEF3T2yfT1O' --output covid.test.csv

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=19CCyCgJrUxtvgZF53vnctJiOJ23T5mqF
To: /content/covid.train.csv
100% 2.00M/2.00M [00:00<00:00, 207MB/s]
/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1CE240jLm2npU-tdz81-oVKEF3T2yfT1O
To: /content/covid.test.csv
100% 651k/651k [00:00<00:00, 144MB/s]


Import Some Packages

In [5]:
# PyTorch
import torch
# torch:PyTorch 的核心庫，用於實現張量運算和深度學習模型的開發。
import torch.nn as nn
# torch.nn:PyTorch 的神經網絡模組，提供了各種神經網絡層和模型構建的工具。
from torch.utils.data import Dataset, DataLoader
# torch.utils.data.Dataset:提供一個基礎類別，用於定義和加載自訂數據集。
# torch.utils.data.DataLoader:用於將 Dataset 包裝成可迭代的數據加載器，支持批量處理、隨機打亂數據等功能。

# For data preprocess
import numpy as np
# numpy:用於進行高效的數值運算，例如陣列操作。
import csv
# csv:用於處理 CSV 文件格式的數據。
import os
# os:提供操作系統功能，例如檔案和目錄操作。

# For plotting
import matplotlib.pyplot as plt
# matplotlib.pyplot:用於資料視覺化，繪製圖表（如折線圖、散點圖等）。
from matplotlib.pyplot import figure
# figure:用於建立新的圖表畫布，方便控制圖的尺寸。

myseed = 42069  # set a random seed for reproducibility
# myseed = 42069:自訂的隨機種子值，可以選擇任何整數。隨機種子可以讓每次運行程式時的結果保持一樣。
torch.backends.cudnn.deterministic = True
# torch.backends.cudnn.deterministic = True:強制使用確定性的操作，保證相同輸入下產生相同的輸出。開啟後會略微降低執行效能。
torch.backends.cudnn.benchmark = False
# torch.backends.cudnn.benchmark = False:禁止動態調整最佳內核算法，這樣可以保證結果穩定。
np.random.seed(myseed)
# np.random.seed(myseed):設置 NumPy 的隨機數種子，確保數據處理過程的穩定性。
torch.manual_seed(myseed)
# torch.manual_seed(myseed):設置 PyTorch 的隨機數種子，用於 CPU 運算。
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(myseed)
    # torch.cuda.manual_seed_all(myseed):如果 GPU 可用，為所有 GPU 設置隨機數種子。

Some Utilities

In [6]:
def get_device():
    ''' Get device (if GPU is available, use GPU) '''
    return 'cuda' if torch.cuda.is_available() else 'cpu'
# 這個函數用於檢查當前的硬體環境是否支持 GPU。如果支持，則返回 'cuda'（代表使用 GPU）；如果不支持，則返回 'cpu'（代表使用 CPU）
# torch.cuda.is_available():PyTorch 提供的函數，用於檢測是否有可用的 GPU。如果有可用的 GPU 並且已正確安裝 CUDA，返回 True；否則返回 False。

def plot_learning_curve(loss_record, title=''):
  # 這個函數用於繪製神經網絡訓練過程中的學習曲線，包括訓練損失（train loss）和驗證損失（dev loss）。
  # loss_record:一個字典，包含兩個鍵：'train': 訓練過程中每個步驟的損失值列表。'dev': 驗證過程中每個步驟的損失值列表。
  # title:學習曲線的標題，可選，默認為空字串。
    ''' Plot learning curve of your DNN (train & dev loss) '''
    total_steps = len(loss_record['train'])
    x_1 = range(total_steps)
    # 訓練損失的橫軸對應的是總步驟數。
    x_2 = x_1[::len(loss_record['train']) // len(loss_record['dev'])]
    # 驗證損失通常記錄的頻率較低，這行程式碼將其橫軸按比例對齊到訓練損失。
    # len(loss_record['train']):表示訓練損失記錄的總步數(總數據點數量)。
    # len(loss_record['dev']):表示驗證損失記錄的總步數(總數據點數量)。驗證損失通常記錄的頻率低於訓練損失，因此這個數值通常小於 len(loss_record['train'])。
    # len(loss_record['train']) // len(loss_record['dev']):計算訓練損失與驗證損失的比例，得到一個整數，表示每隔多少個訓練步驟記錄一次驗證損失。 A // B表示A除以B
    # x_1[::步進值]:通過切片操作從 x_1 中按指定的間隔選取值。步進值 = len(loss_record['train']) // len(loss_record['dev'])，確保驗證損失的橫軸與訓練損失對齊。
    figure(figsize=(6, 4))
    plt.plot(x_1, loss_record['train'], c='tab:red', label='train')
    # 訓練損失(紅色)
    plt.plot(x_2, loss_record['dev'], c='tab:cyan', label='dev')
    # 驗證損失(藍色)
    plt.ylim(0.0, 5.)
    plt.xlabel('Training steps')
    plt.ylabel('MSE loss')
    plt.title('Learning curve of {}'.format(title))
    plt.legend()
    plt.show()
    # 圖表設定


def plot_pred(dv_set, model, device, lim=35., preds=None, targets=None):
  # plot_pred 函數：接收驗證集 dv_set、模型 model、計算設備 device，以及預測值和目標值(可選)。
  # 如果未提供預測值和目標值，函數會利用模型和驗證集計算它們。
  # 生成一個散點圖，其中每個點表示一個樣本的真實值和模型預測值。
  # dv_set:驗證數據集(通常是 PyTorch 的 DataLoader)，包含樣本特徵和對應的真實目標值。
  # model:預訓練好的深度神經網絡模型，用於計算預測值。
  # device:設備類型（如 'cuda' 或 'cpu'），決定數據和模型運算的硬體。
  # lim:圖表的最大範圍(上面設定 35)，用於設定 x 和 y 軸的範圍。
  # preds 和 targets:預測值和真實值。如果已經計算好，可以直接傳入以繪圖；否則函數會計算它們。
    ''' Plot prediction of your DNN '''
    if preds is None or targets is None:
    # 檢查是否已經提供了預測值 preds 和目標值 targets。
    # 如果這兩者之一為空，表示需要從驗證數據集和模型中計算它們。
        model.eval()
        # model.eval()：進入評估模式，禁用 Dropout 等隨機操作，確保模型行為穩定。
        preds, targets = [], []
        # 用於存放模型對每批驗證數據的預測結果(preds)和真實值(targets)
        for x, y in dv_set:
        # dv_set 通常是 PyTorch 的 DataLoader，每次提供一批數據。
        # x 是樣本的特徵，y 是對應的真實標籤(目標值)。
            x, y = x.to(device), y.to(device)
            # 使用 to(device) 方法將數據移動到指定的計算設備上(例如 CPU 或 GPU)。
            # 保證數據和模型在同一設備上，避免計算錯誤。
            with torch.no_grad():
            # torch.no_grad()：避免儲存中間結果的梯度，節省記憶體和計算資源。
                pred = model(x)
                # 將一批特徵 x 傳入模型，得到對應的預測值 pred。
                preds.append(pred.detach().cpu())
                # detach().cpu()：從計算圖中分離張量，移動到 CPU，方便後續處理。
                # detach()：從計算圖中分離張量，避免影響後續操作。
                # cpu()：將張量移動到 CPU，方便後續處理。
                # 使用 append 方法將每批結果添加到列表中。
                targets.append(y.detach().cpu())
        preds = torch.cat(preds, dim=0).numpy()
        targets = torch.cat(targets, dim=0).numpy()
        # 將 preds 中的所有批次結果在第一個維度(樣本數量維度)上拼接，形成一個完整的張量。
        # .numpy()：將 PyTorch 張量轉換為 NumPy 陣列，方便進一步操作或繪圖。

    figure(figsize=(5, 5))
    plt.scatter(targets, preds, c='r', alpha=0.5)
    # 每個點的 x 座標是目標值(真實值 targets)，y 座標是模型的預測值(preds)。
    # 點的顏色為紅色，透明度設為 0.5。
    plt.plot([-0.2, lim], [-0.2, lim], c='b')
    # 繪製一條藍色對角線，表示理想情況下預測值與目標值完全一致的參考線。
    plt.xlim(-0.2, lim)
    plt.ylim(-0.2, lim)
    # 設定 x 和 y 軸範圍，方便比較。
    plt.xlabel('ground truth value')
    plt.ylabel('predicted value')
    plt.title('Ground Truth v.s. Prediction')
    plt.show()

# Preprocess

We have three kinds of datasets:\
train: for training\
dev: for validation\
test: for testing (w/o target value)

# Dataset

The COVID19Dataset below does:\
read .csv files\
extract features\
split covid.train.csv into train/dev sets\
normalize features\
Finishing TODO below might make you pass medium baseline.

In [7]:
class COVID19Dataset(Dataset):
    ''' Dataset for loading and preprocessing the COVID19 dataset '''
    # COVID19Dataset 是一個繼承自 PyTorch Dataset 的類別，目的是實現資料加載和處理的自定義邏輯。
    # 作用：將數據集從文件中讀取到內存。進行必要的數據預處理，方便後續模型使用。
    def __init__(self,
                 path, # 資料集文件的路徑，例如 covid.train.csv 或 covid.test.csv。
                 mode='train', # 指定數據集模式（訓練或測試）。這可能影響數據處理方式，例如是否包含目標值(標籤)。
                 target_only=False): # 用於選擇是否只使用部分特徵。
                                     # 預設為 False，表示使用所有特徵。
                                     # 當為 True 時，會根據特定邏輯(如代碼中的 TODO 部分)只選擇部分特徵。
        self.mode = mode

        # Read data into numpy arrays
        with open(path, 'r') as fp:
            data = list(csv.reader(fp))
            # 將所有行存入列表 data 中，包含表頭和數據。
            # csv.reader(fp)：讀取 CSV 文件的內容，每行作為一個列表。
            data = np.array(data[1:])[:, 1:].astype(float)
            # data[1:]：跳過第一行(表頭)。
            # [:, 1:]：跳過第一列(通常是樣本 ID 或其他無關的識別符)。
            # .astype(float)：將數據轉換為浮點數類型，方便後續數值運算。

        if not target_only:
        # 當 target_only 為 False 時，選擇所有特徵。
            feats = list(range(93))
            # 建立包含 0 到 92 的索引列表，表示選取數據的所有 93 個特徵。
        else:
        # 當 target_only 為 True 時，僅選擇特定特徵（未實現）。
            # TODO: Using 40 states & 2 tested_positive features (indices = 57 & 75)
            pass

        if mode == 'test':
            # Testing data
            # data: 893 x 93 (40 states + day 1 (18) + day 2 (18) + day 3 (17))
            data = data[:, feats]
            # 使用 feats 選取特定的特徵列。此處的特徵列數可能是 93 列，包含 40 個狀態特徵與 3 天的特徵。
            # feats：是一個包含列索引的列表或數組，這些列索引用於選擇需要的特徵。
            # 例如，如果 feats = [0, 2, 5]，那麼 data[:, feats] 就會選擇原數據中的第 0 列、第 2 列和第 5 列作為新數據的特徵。
            # :（冒號）：這是 numpy 中的切片操作符，表示選擇所有行。因此，data[:, feats] 表示選擇所有樣本的指定列。
            self.data = torch.FloatTensor(data)
            # 將 NumPy 陣列轉換為 PyTorch 的浮點張量。
            # 結果：self.data 是一個 PyTorch 張量，用於後續的模型輸入。
        else:
            # Training data (train/dev sets)
            # data: 2700 x 94 (40 states + day 1 (18) + day 2 (18) + day 3 (18))
            target = data[:, -1]
            # 提取目標值列，存入變數 target。
            data = data[:, feats]
            # 根據 feats 選取特定的特徵列。

            # Splitting training data into train & dev sets
            if mode == 'train':
            # mode == 'train':這個條件判斷是用來確定當前處於訓練模式。如果 mode 的值為 'train'，則進行後續操作。
                indices = [i for i in range(len(data)) if i % 10 != 0]
                # range(len(data)) 會產生一個範圍，該範圍包含了數據集 data 中所有樣本的索引。假設 data 有 2700 筆樣本，這段程式碼會生成從 0 到 2699 的索引範圍。
                # i % 10 != 0 是一個條件過濾表達式，表示選擇那些 i (索引值) 不是 10 的倍數的樣本。
                # i % 10 是對索引 i 做除以 10 的餘數運算，如果結果不是 0，則 i 不是 10 的倍數，該樣本會被選中。
                # 選擇 90% 的數據用於訓練。
            elif mode == 'dev':
                indices = [i for i in range(len(data)) if i % 10 == 0]
                # 選擇 10% 的數據用於驗證。

            # Convert data into PyTorch tensors
            self.data = torch.FloatTensor(data[indices])
            # torch.FloatTensor：將過濾後的數據和目標值轉換為 PyTorch 浮點張量，便於後續用於模型訓練或評估。
            # data[indices]：根據選定的索引過濾數據，僅保留訓練或驗證所需的樣本。
            self.target = torch.FloatTensor(target[indices])

        # Normalize features (you may remove this part to see what will happen)
        self.data[:, 40:] = \
        # self.data 是一個包含所有特徵的張量。這一行程式碼選擇了 self.data 中第 40 列及之後的所有列（即特徵），進行標準化處理。
            (self.data[:, 40:] - self.data[:, 40:].mean(dim=0, keepdim=True)) \
            # 這一步是將每列的數據減去該列的均值，這樣每列的均值變為 0。
            / self.data[:, 40:].std(dim=0, keepdim=True)
            # 然後用每列的標準差對每個數據點進行除法，這樣每列的標準差變為 1。
            # dim=0：這個參數表示沿著列(第 0 維)計算均值和標準差，即每列的均值和標準差。
            # keepdim=True：這個參數保證了輸出的均值和標準差維度不會被簡化，保持與輸入數據相同的維度。

        self.dim = self.data.shape[1]
        # self.data.shape[1] 返回 self.data 的列數（即每個樣本的特徵數量），並將其賦值給 self.dim。
        # self.dim 用來記錄數據集中特徵的數量。

        print('Finished reading the {} set of COVID19 Dataset ({} samples found, each dim = {})'
              .format(mode, len(self.data), self.dim))
        # mode：這是數據集的模式（'train'、'dev' 或 'test'），表示當前載入的是訓練集、驗證集還是測試集。
        # len(self.data)：len(self.data) 返回數據集中樣本的數量。

    def __getitem__(self, index):
    # __getitem__ 是 Python 中的一個特殊方法，它允許對象像列表一樣進行索引。當我們使用 dataset[index] 訪問數據集中的某個樣本時，實際上會調用這個方法。
        # Returns one sample at a time
        if self.mode in ['train', 'dev']:
        # self.mode 是一個標誌，表示當前的數據集處於哪個模式，可能的值有 'train'(訓練集)、'dev'(驗證集)或 'test'(測試集)。
            # For training
            return self.data[index], self.target[index]
        else:
            # For testing (no target)
            return self.data[index]

    def __len__(self):
    # __len__ 是 Python 中的特殊方法，用來返回對象的長度。當我們對一個數據集（例如 dataset）調用 len(dataset) 時，Python 會自動調用這個方法。
    # 在 PyTorch 中，Dataset 類別需要實現這個方法來告訴 DataLoader 這個數據集有多少個樣本。
        # Returns the size of the dataset
        return len(self.data)
        # self.data 是在 COVID19Dataset 類別中加載並處理過的特徵數據。
        # self.data 是一個包含數據的 torch.Tensor，表示每個樣本的特徵。
        # len(self.data) 返回 self.data 張量中的樣本數量(即數據集的大小)。
        # 如果 self.data 中有 2700 個樣本，則 len(self.data) 返回 2700。

SyntaxError: invalid syntax (<ipython-input-7-c9d070cbf66a>, line 71)

# DataLoader

A DataLoader loads data from a given Dataset into batches.

In [8]:
def prep_dataloader(path, mode, batch_size, n_jobs=0, target_only=False):
# path：指定數據集文件的路徑(如 CSV 文件)，用來初始化 COVID19Dataset。
# mode：指定數據集的模式，可以是 'train'(訓練集)、'dev'(驗證集)或 'test'(測試集)。這個參數會影響數據集的處理方式。
# batch_size：指定每個批次的樣本數量。DataLoader 會按照這個數量將數據分批。
# n_jobs（默認值為 0）：指定加載數據時使用的工作進程數量。n_jobs=0 表示不使用多進程。
# target_only（默認值為 False）：指定是否只加載目標值。如果設為 True，則只返回目標值；如果設為 False，則返回特徵和目標值。
    ''' Generates a dataset, then is put into a dataloader. '''
    dataset = COVID19Dataset(path, mode=mode, target_only=target_only)  # Construct dataset
    # 使用傳入的 path、mode 和 target_only 參數創建 COVID19Dataset 實例。COVID19Dataset 是自定義的數據集類別，它會讀取並處理數據。
    # path：數據集的路徑。
    # mode：設置數據集的模式（訓練、驗證或測試）。
    # target_only：決定是否只加載目標值。
    dataloader = DataLoader(
    # DataLoader 是 PyTorch 提供的數據加載器，用於將數據集分成批次並有效地加載數據。
        dataset, batch_size, # dataset：將創建的 COVID19Dataset 作為數據集。
                             # batch_size：設置每個批次的樣本數量。
        shuffle=(mode == 'train'), drop_last=False, # shuffle=(mode == 'train')：當 mode 是 'train' 時，設置為 True，表示在每次迭代時打亂數據；如果是驗證或測試模式，則不打亂數據（shuffle=False）。
                                                    # drop_last=False：這表示如果批次中的樣本數量不足以填滿最後一個批次，則仍然包含這個批次。如果設置為 True，則丟棄最後一個不完整的批次。
        num_workers=n_jobs, pin_memory=True)                            # Construct dataloader
        # num_workers=n_jobs：指定用於數據加載的工作進程數量。n_jobs=0 表示使用單進程加載數據。
        # pin_memory=True：設置為 True，這樣會將數據加載到鎖頁內存中，這樣可以加速將數據從 CPU 複製到 GPU 的過程，對於 GPU 加速訓練有幫助。
    return dataloader

# Deep Neural Network

NeuralNet is an nn.Module designed for regression. The DNN consists of 2 fully-connected layers with ReLU activation. This module also included a function cal_loss for calculating loss.

In [10]:
class NeuralNet(nn.Module):
# NeuralNet 是一個簡單的全連接神經網絡。
# 繼承自 nn.Module，使得這個類別可以使用 PyTorch 提供的各種工具（如自動微分、權重管理等）。
    ''' A simple fully-connected deep neural network '''
    def __init__(self, input_dim):
    # 定義了神經網絡的結構，接收輸入特徵數量 input_dim 作為參數。
        super(NeuralNet, self).__init__()
        # 初始化父類（nn.Module），使得 PyTorch 能正確地追蹤這個模型中的參數。

        # Define your neural network here
        # TODO: How to modify this model to achieve better performance?
        self.net = nn.Sequential( # nn.Sequential：將多層操作按順序組合起來，簡化了模型的定義。
            nn.Linear(input_dim, 64), # 全連接層，nn.Linear(in_features, out_features)，將輸入的維度映射到指定的輸出維度。
                                      # 第一層：輸入維度 input_dim，輸出維度 64。
            nn.ReLU(), # 激活函數，將輸入的負值置為 0，保留正值，具有良好的梯度傳播效果。
            nn.Linear(64, 1) # 第二層：輸入維度 64，輸出維度 1。
        )

        # Mean squared error loss
        self.criterion = nn.MSELoss(reduction='mean')
        # nn.MSELoss：均方誤差損失函數，用於回歸任務。
        # reduction='mean'：計算損失時，對所有樣本的損失值取平均。

    def forward(self, x):
    # forward(self, x)： 這是神經網路中的前向傳播方法，用於定義輸入數據如何通過模型進行計算以產生輸出。該方法通常實現在繼承自 torch.nn.Module 的自定義類別中。
    # x：輸入的張量 (tensor)，形狀為 (batch_size, input_dim)，其中 batch_size 是批量大小，input_dim 是輸入的特徵數。
        ''' Given input of size (batch_size x input_dim), compute output of the network '''
        return self.net(x).squeeze(1)
        # self.net(x)：將輸入 x 傳遞給網路 self.net。self.net 是在類別中定義的子網路，可能由多個層組成（例如線性層或激活函數）。
        # squeeze(1)：移除輸出張量中維度大小為 1 的第二個維度。例如：如果輸出形狀為 (batch_size, 1)，則經過 .squeeze(1) 後，形狀會變成 (batch_size,)。

    def cal_loss(self, pred, target):
    # 這是用於計算損失的函式，根據模型的預測值 (pred) 和目標值 (target) 計算損失值。
    # pred：模型的預測輸出，通常是張量(tensor)。
    # target：真實標籤或目標值，形狀與 pred 通常相同。

        ''' Calculate loss '''
        # TODO: you may implement L1/L2 regularization here
        return self.criterion(pred, target)
        # self.criterion(pred, target)：
        # 使用 self.criterion 計算損失，self.criterion 通常是 PyTorch 中定義的損失函式，例如：
        # MSELoss（均方誤差，適用於回歸問題）。
        # CrossEntropyLoss（交叉熵損失，適用於分類問題）。
        # BCELoss（二元交叉熵損失，適用於二元分類）。

# Train/Dev/Test

In [11]:
def train(tr_set, dv_set, model, config, device):
# 這是用於訓練深度神經網路 (Deep Neural Network, DNN) 的函式。
# tr_set: 訓練數據集。
# dv_set: 驗證數據集（用於驗證模型表現，非訓練用）。
# model: 欲訓練的深度學習模型。
# config: 包含訓練相關超參數（如學習率、優化器類型等）的設定。
# device: 設定模型運行的設備（如 cpu 或 cuda）。
    ''' DNN training '''

    n_epochs = config['n_epochs']  # Maximum number of epochs
    # 從 config 中提取最大訓練輪數，確保訓練過程有一個終止條件。

    # Setup optimizer
    optimizer = getattr(torch.optim, config['optimizer'])(
        model.parameters(), **config['optim_hparas'])

    min_mse = 1000.
    loss_record = {'train': [], 'dev': []}      # for recording training loss
    early_stop_cnt = 0
    epoch = 0
    while epoch < n_epochs:
        model.train()                           # set model to training mode
        for x, y in tr_set:                     # iterate through the dataloader
            optimizer.zero_grad()               # set gradient to zero
            x, y = x.to(device), y.to(device)   # move data to device (cpu/cuda)
            pred = model(x)                     # forward pass (compute output)
            mse_loss = model.cal_loss(pred, y)  # compute loss
            mse_loss.backward()                 # compute gradient (backpropagation)
            optimizer.step()                    # update model with optimizer
            loss_record['train'].append(mse_loss.detach().cpu().item())

        # After each epoch, test your model on the validation (development) set.
        dev_mse = dev(dv_set, model, device)
        if dev_mse < min_mse:
            # Save model if your model improved
            min_mse = dev_mse
            print('Saving model (epoch = {:4d}, loss = {:.4f})'
                .format(epoch + 1, min_mse))
            torch.save(model.state_dict(), config['save_path'])  # Save model to specified path
            early_stop_cnt = 0
        else:
            early_stop_cnt += 1

        epoch += 1
        loss_record['dev'].append(dev_mse)
        if early_stop_cnt > config['early_stop']:
            # Stop training if your model stops improving for "config['early_stop']" epochs.
            break

    print('Finished training after {} epochs'.format(epoch))
    return min_mse, loss_record

# Validation

In [12]:
def dev(dv_set, model, device):
    model.eval()                                # set model to evalutation mode
    total_loss = 0
    for x, y in dv_set:                         # iterate through the dataloader
        x, y = x.to(device), y.to(device)       # move data to device (cpu/cuda)
        with torch.no_grad():                   # disable gradient calculation
            pred = model(x)                     # forward pass (compute output)
            mse_loss = model.cal_loss(pred, y)  # compute loss
        total_loss += mse_loss.detach().cpu().item() * len(x)  # accumulate loss
    total_loss = total_loss / len(dv_set.dataset)              # compute averaged loss

    return total_loss

# Testing

In [13]:
def test(tt_set, model, device):
    model.eval()                                # set model to evalutation mode
    preds = []
    for x in tt_set:                            # iterate through the dataloader
        x = x.to(device)                        # move data to device (cpu/cuda)
        with torch.no_grad():                   # disable gradient calculation
            pred = model(x)                     # forward pass (compute output)
            preds.append(pred.detach().cpu())   # collect prediction
    preds = torch.cat(preds, dim=0).numpy()     # concatenate all predictions and convert to a numpy array
    return preds

# Setup Hyper-parameters

config contains hyper-parameters for training and the path to save your model.

In [14]:
device = get_device()                 # get the current available device ('cpu' or 'cuda')
os.makedirs('models', exist_ok=True)  # The trained model will be saved to ./models/
target_only = False                   # TODO: Using 40 states & 2 tested_positive features

# TODO: How to tune these hyper-parameters to improve your model's performance?
config = {
    'n_epochs': 3000,                # maximum number of epochs
    'batch_size': 270,               # mini-batch size for dataloader
    'optimizer': 'SGD',              # optimization algorithm (optimizer in torch.optim)
    'optim_hparas': {                # hyper-parameters for the optimizer (depends on which optimizer you are using)
        'lr': 0.001,                 # learning rate of SGD
        'momentum': 0.9              # momentum for SGD
    },
    'early_stop': 200,               # early stopping epochs (the number epochs since your model's last improvement)
    'save_path': 'models/model.pth'  # your model will be saved here
}

# Load data and model

In [15]:
tr_set = prep_dataloader(tr_path, 'train', config['batch_size'], target_only=target_only)
dv_set = prep_dataloader(tr_path, 'dev', config['batch_size'], target_only=target_only)
tt_set = prep_dataloader(tt_path, 'test', config['batch_size'], target_only=target_only)

NameError: name 'COVID19Dataset' is not defined

# Start Training!

In [16]:
model_loss, model_loss_record = train(tr_set, dv_set, model, config, device)

NameError: name 'tr_set' is not defined

# Testing

The predictions of your model on testing set will be stored at pred.csv.

In [ ]:
def save_pred(preds, file):
    ''' Save predictions to specified file '''
    print('Saving results to {}'.format(file))
    with open(file, 'w') as fp:
        writer = csv.writer(fp)
        writer.writerow(['id', 'tested_positive'])
        for i, p in enumerate(preds):
            writer.writerow([i, p])

preds = test(tt_set, model, device)  # predict COVID-19 cases with your model
save_pred(preds, 'pred.csv')         # save prediction file to pred.csv

# Hints

# Simple Baseline

Run sample code
# Medium Baseline

Feature selection: 40 states + 2 tested_positive (TODO in dataset)
# Strong Baseline

Feature selection (what other features are useful?)\
DNN architecture (layers? dimension? activation function?)\
Training (mini-batch? optimizer? learning rate?)\
L2 regularization\
There are some mistakes in the sample code, can you find them?